# Web-Summary Notebook — LM Studio
_A tiny utility to fetch a page → extract text → ask the model for a short summary._

### Imports

In [1]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

### OpenAI-compatible client

In [2]:
# Configure OpenAI-compatible client (GLM endpoint)
client = OpenAI(
  base_url="http://localhost:1234/v1",
)

In [3]:
# quick smoke test (optional)
message = "Hello, Llama 3.2! This is my first ever message to you! Hi!"
response = client.chat.completions.create(model="llama-3.2-3b-instruct", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! It's great to meet you! I'm excited to be your first conversation partner. How can I assist you today? Do you have any questions, topics you'd like to discuss, or just want to chat? I'm all ears (or rather, all text)!


### Web fetcher

In [4]:
# Some websites need proper headers when fetching them
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """Create this Website object from the given url using BeautifulSoup."""
        self.url = url
        response = requests.get(url, headers=headers, timeout=20)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        # strip junk
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True) if soup.body else ""

In [5]:
rp = Website("https://rp30.us")
print(rp.title)
print(rp.text)

Rohith's Portfolio
RP.
Home
Skills
Experience
Education
Projects
Contact
Home
Skills
Experience
Education
Projects
Contact
Rohith Goud Panjala
Full-Stack Software Engineer & Problem Solver
Engineering Scalable & Impactful Solutions
I'm a Full-Stack Software Engineer who builds fast, scalable, and intelligent systems that solve real-world problems. I thrive on turning complexity into clarity by designing solutions that are clean, efficient, and built to last. With a sharp eye for detail and a passion for technology, I bring energy and ownership to every project I touch. I don't just code. I deliver outcomes, move fast, and grow with every challenge.
Resume
Get In Touch
Scroll
My Skills
Technical expertise and tools I work with
Scroll
Work Experience
My professional journey and key contributions
Jun 2024 - Aug 2024
Software Engineer Intern
ZROin
Remote (Baltimore, MD)
Engineered scalable backend services in Nest.js, enhancing ZROin's platform for 50+ small businesses by integrating 3D st

### Prompt builders

In [6]:
# system prompt — tweak freely
system_prompt = (
    "You are an assistant that analyzes the contents of a website "
    "and provides a short summary, ignoring navigation text. "
    "Respond in markdown."
)

def user_prompt_for(website: Website):
    up = f"You are looking at a website titled {website.title}\n"
    up += "The contents of this website is as follows:\n"
    up += "Please provide a short summary of this website in markdown.\n"
    up += "If it includes news or announcements, summarize these too.\n\n"
    up += website.text
    return up

def messages_for(website: Website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [7]:
print(user_prompt_for(rp))

You are looking at a website titled Rohith's Portfolio
The contents of this website is as follows:
Please provide a short summary of this website in markdown.
If it includes news or announcements, summarize these too.

RP.
Home
Skills
Experience
Education
Projects
Contact
Home
Skills
Experience
Education
Projects
Contact
Rohith Goud Panjala
Full-Stack Software Engineer & Problem Solver
Engineering Scalable & Impactful Solutions
I'm a Full-Stack Software Engineer who builds fast, scalable, and intelligent systems that solve real-world problems. I thrive on turning complexity into clarity by designing solutions that are clean, efficient, and built to last. With a sharp eye for detail and a passion for technology, I bring energy and ownership to every project I touch. I don't just code. I deliver outcomes, move fast, and grow with every challenge.
Resume
Get In Touch
Scroll
My Skills
Technical expertise and tools I work with
Scroll
Work Experience
My professional journey and key contribut

## My Runs
List any sites you want summarized below and run the next cell.

### Model call + display helpers

In [8]:
def summarize(url):
    website = Website(url)
    response = client.chat.completions.create(
        model = "llama-3.2-3b-instruct",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [9]:
summarize("https://rp30.us")

"# Rohith's Portfolio Summary\n==========================\n\n**Overview**\n---------------\n\nRohith's portfolio showcases his experience as a Full-Stack Software Engineer & Problem Solver. He has worked on various projects, including scalable backend services, real-time backend development, and ethical software research.\n\n### Education\n\n* Master of Professional Studies in Software Engineering (2023-2025), University of Maryland Baltimore County\n* Bachelor of Technology in Electronics and Communication Engineering (2017-2021), Guru Nanak Institutions (GNI)\n\n### Work Experience\n\n#### Jun 2024 - Aug 2024: Software Engineer Intern, ZROin\nEngineered scalable backend services using Nest.js, enhancing the platform for small businesses.\n\n#### Jun 2024 - Aug 2024: Backend Developer Intern, Reel Talk\nDeveloped a real-time backend for web, iOS, and Android platforms using Firebase, GCP, Node.js, and Express.js.\n\n#### Sep 2023 - Apr 2024: Research Assistant, The Ethical Software La

In [10]:
# A function to display this nicely in the Jupyter output, using markdown
display_summary("https://rp30.us")

**Rohith's Portfolio**
=====================================

### Overview

Rohith's portfolio showcases his expertise as a Full-Stack Software Engineer, highlighting his technical skills and accomplishments in various projects.

### Recent News/Announcements

None mentioned on the website.

### Summary of Content

The website is divided into sections:

*   **Introduction**: A brief introduction to Rohith, highlighting his role as a Full-Stack Software Engineer.
*   **Skills**: Technical expertise and tools used by Rohith.
*   **Experience**: Work experience section with achievements in software engineering internships at ZROin, Reel Talk, and The Ethical Software Lab, UMBC.
*   **Education**: Academic background, including his Master of Professional Studies in Software Engineering from the University of Maryland Baltimore County.
*   **Projects**: Showcase of technical expertise and problem-solving abilities through capstone projects like CRWLR - Terms of Service & Privacy Policy Analyzer.

### Projects

CRWLR is a full-stack cloud-native platform for crawling, summarizing, and analyzing legal documents using AI and NLP. This project demonstrates Rohith's ability to:

*   Design robust solutions
*   Integrate AI APIs effectively
*   Build secure document processing pipelines
*   Optimize performance

Rohith's portfolio highlights his expertise as a Full-Stack Software Engineer, with a strong focus on problem-solving and technical skills.

### Technical Details

*   Technologies used: Next.js 15, FastAPI, Tailwind CSS, Gemini API, Clerk, Typesense, Playwright, BeautifulSoup, GitHub Actions, Google Cloud Run, Vercel.
*   Frontend framework: Next.js 15 with Tailwind CSS
*   Backend services: Robust backend services built using FastAPI featuring Gemini API integration for AI-driven summarization

### Contact Information

Rohith can be reached through his email (rohithgp30@gmail.com) or connected on LinkedIn and GitHub.

### Let's try more websites

In [11]:
display_summary("https://cnn.com")

BadRequestError: Error code: 400 - {'error': 'Trying to keep the first 4575 tokens when context the overflows. However, the model is loaded with context length of only 4096 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}

In [12]:
display_summary("https://anthropic.com")

# Anthropic Website Summary
=====================================

**About**
-----------

Anthropic is a public benefit corporation dedicated to securing the benefits and mitigating the risks of AI. They aim to build tools that serve humanity's long-term well-being.

**Research Focus**
-----------------

Anthropic focuses on building responsible AI technologies, such as Claude, which is an agent for coding and computer use. Their research emphasizes safety at the frontier, exploring topics like agentic misalignment, alignment, and interpretability.

**News and Announcements**
-------------------------

*   **Claude Sonnet 4.5**: The best model in the world for agents, coding, and computer use.
*   **Claude Opus 4.1**: A new product release.
*   **Project Vend**: An announcement on a new project (August 2025)
*   **Tracing the thoughts of a large language model**: An announcement on interpretability research (March 2025)

**Products**
------------

Anthropic offers various products, including:

*   Claude: A powerful agent for coding and computer use.
*   Claude Code: A developer platform for building with Claude.
*   Max plan, Team plan, Enterprise plan: Pricing plans for different users.

**Company Information**
----------------------

Anthropic is a company dedicated to responsible AI development. They have an **Anthropic Academy** where users can learn to build with Claude, and they offer various programs like the **Startups program**.

In [14]:
display_summary("https://openai.com")

# OpenAI Website Summary
=====================================

The OpenAI website provides an overview of the company's research, products, and services. The main sections include:

*   **Research**: Details on various AI-related projects and advancements, such as GPT-5, Sora 2, and OpenAI o3.
*   **Safety**: Information on security measures and transparency in OpenAI's models.
*   **For Business**: Solutions for businesses, including enterprise, education, and API platforms.
*   **About Us**: Company history, charter, careers, and brand guidelines.

## Recent News Announcements

### Upcoming Events

*   **OpenAI DevDay 2025**: Catch up on recent announcements.
*   **Sora 2 is here**: A new version of Sora was released on September 30, 2025.

### Product Updates

*   **Codex is now generally available**: Codex is a conversational AI model that can be integrated into various applications.
*   **AgentKit, new Evals, and RFT for agents**: AgentKit provides tools for creating more effective agents in chatbots.
*   **Buy it in ChatGPT: Instant Checkout and the Agentic Commerce Protocol**: A new feature allowing users to purchase items directly within ChatGPT.

### Partnerships and Collaborations

*   **AMD and OpenAI announce strategic partnership to deploy 6 gigawatts of AMD GPUs**: A partnership aimed at accelerating AI research and development.

## Publications

The website also features articles on various AI-related topics, including:

*   Measuring the performance of OpenAI's models on real-world tasks.
*   Detecting and reducing scheming in AI models.

These publications provide insights into OpenAI's approaches to developing safe and effective AI models.